In [1]:
## importing libraries 
import numpy as np
import scipy as sc
import pandas as pd
from scipy.stats import beta,norm
from math import ceil
import random
from scipy.stats import uniform
import pymc3
from bokeh.charts import output_notebook, show
import DBDA2E_utilities

In [2]:
output_notebook()

Loading BokehJS ...

## Exercise 7.1
## Purpose : To Experiment with  Metropois Algorithm as displayed in figure7.4

In [3]:

myData = np.concatenate((np.repeat(0,6), np.repeat(1,14)))

# Define the Bernoulli likelihood function, p(D|theta).
# The argument theta could be a vector, not just a scalar.
def likelihood(theta,data):
    z = sum(data)
    N = len(data)
    pDataGivenTheta = [(theta**z) * (1-theta)**(N-z),0][theta > 1 or theta <0]
    # The theta values passed into this function are generated at random,
    # and therefore might be inadvertently greater than 1 or less than 0.
    # The likelihood for theta > 1 or for theta < 0 is zero:
    return(pDataGivenTheta)
# Define the prior density function. 
def prior(theta):
    pTheta = [beta.pdf(theta,1,1),0][theta > 1 or theta <0]
    # The theta values passed into this function are generated at random,
    # and therefore might be inadvertently greater than 1 or less than 0.
    # The prior for theta > 1 or for theta < 0 is zero:
    return pTheta

# Define the relative probability of the target distribution, 
# as a function of vector theta. For our application, this
# target distribution is the unnormalized posterior distribution.
def targetRelProb(theta,data):
    targetRelProb = likelihood(theta, data) * prior(theta)
    return(targetRelProb)


# Specify the length of the trajectory, i.e., the number of jumps to try:
trajLength = 50000 # arbitrary large number
trajectory = np.repeat(0.0,trajLength)  ## Note 0.0 to ensure float arrays
trajectory[0] = 0.01 # arbitrary value

burnIn = ceil( 0.0 * trajLength ) # arbitrary number, less than trajLength
# Initialize accepted, rejected counters, just to monitor performance:
nAccepted = 0
nRejected = 0


In [4]:
# Now generate the random walk. The 't' index is time or trial in the walk.
# Specify seed to reproduce same random walk:
random.seed(47405)
proposalSD = np.array([0.02,0.2,2.0])[2]

for t in range(trajLength -1):
    currentPosition = trajectory[t]
    # Use the proposal distribution to generate a proposed jump.
    proposedJump = norm.rvs(loc = 0, scale = proposalSD)
    probAccept = min(1, targetRelProb(currentPosition + proposedJump,myData )/ targetRelProb( currentPosition , myData ))
    if uniform.rvs() < probAccept:
        # accept the proposed jump
        trajectory[t+1] = currentPosition + proposedJump
        # increment the accepted counter, just to monitor performance
        nAccepted = [nAccepted+1,nAccepted][t > burnIn]
    else:
        # reject the proposed jump, stay at current position
        trajectory[t+1] = currentPosition
        # increment the rejected counter, just to monitor performance
        nRejected = [nRejected+1,nRejected][t > burnIn]
        
# Extract the post-burnIn portion of the trajectory.
acceptedTraj = trajectory[ (burnIn+1) : len(trajectory) ]
# End of Metropolis algorithm.

In [5]:
title = 'Prpsl.SD = {0:.2f} '.format(np.std(acceptedTraj))
show(DBDA2E_utilities.plotPost(acceptedTraj,title= title))




In [6]:
show(DBDA2E_utilities.func_latex())